<a href="https://colab.research.google.com/github/junghodavidlee/Project-3/blob/james-kim/JK_IMDB_Movie_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [14]:
data = pd.read_csv("/content/drive/MyDrive/project-3/IMDB Dataset.csv")

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
# Label encoding
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

In [17]:
data['sentiment'].head()

,sentiment
0,1
1,1
2,1
3,0
4,1


In [18]:
# Split the data into target and features
X = data['review']
y = data['sentiment']
# Do a train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(X_train)

In [20]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=200)

In [21]:
model = keras.Sequential([
    keras.layers.Embedding(input_dim=10000, output_dim=32, input_length=250),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 128s 197ms/step - accuracy: 0.7390 - loss: 0.4895
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 200ms/step - accuracy: 0.8940 - loss: 0.2652
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 123s 197ms/step - accuracy: 0.9098 - loss: 0.2317
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 194ms/step - accuracy: 0.9236 - loss: 0.1963
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 142s 194ms/step - accuracy: 0.9357 - loss: 0.1722


In [23]:
def predict_review(review):
    encoded_review = tokenizer.texts_to_sequences([review])
    padded_review = pad_sequences(encoded_review, maxlen=250, padding='post', truncating='post')
    prediction = model.predict(padded_review)[0][0]
    return "Positive" if prediction > 0.5 else "Negative"

In [24]:
sample_review = "This movie was absolutely fantastic! The storyline was engaging and the characters were well developed."
print(predict_review(sample_review))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Positive


In [25]:
sample_review2 = 'This movie was too long. The story line was all over the place and I was confused the whole time. The ending made no sense either.'
print(predict_review(sample_review2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Positive
